### Celtic Mutations -- From Scratch
### Hidden Markov Model

For my "from scratch" implementation I implemented a standard Hidden Markov Model using
python dictionaries. The final model was able to achieve just under 90% accuracy on a validation
set taken as 10% of provided training data.

*Note: I use state, tag, and label synonymously throughout the code and comments*

This is where all environment variables are declared. **To include test data evaluation, simply
assign the relative filepath to the `test_file` variable.** If the `test_file` variable is an empty
string the test evaluation will be skipped.

The `states` dictionary specifies all the possible states/tags. If one were to be added to the dataset,
it can be included in the model simply by adding the label to this dictionary.

In [54]:
train_file: str = 'data/train.tsv'
test_file: str = ''
val_split: float = 0.1

states: dict = {
    'S': 0,
    'U': 0,
    'T': 0,
    'H': 0,
    'N': 0
}

The `load_data` function constructs sentences by searching for the `<S>` markers.
It'll append each word to a `sentence` list until one of these markers is found. Once that happens, it starts
a new sentence by appending the `sentence` list to the overall `data` list and re-initializes the `sentence`
variable to be filled with the next sentence.

In [41]:
def load_data(file: str) -> list:
    print("Loading data from file {}...".format(file))
    file = open(file, 'r')
    data = []
    sentence = []
    for line in file:
        pieces = line.rstrip("\n").split("\t")
        if pieces[0] == '<S>':
            data.append(sentence)
            sentence = []
        else:
            sentence.append(pieces)
    print("Loaded {} sentences".format(len(data)))
    return data

Here I load the data from the `train_file` using `load_data`, both specified above. The `test_file`
will be loaded only if a filepath was entered.

I then split the training and validation data based on the `val_split` variable. During development
I set this to 0.1 or 0.3, with little difference in accuracy between the two.

In [42]:
train_data: list = load_data(train_file)
if len(test_file) > 0:
    test_data: list = load_data(test_file)
print("Splitting data...")
num_train_samples: int = int(len(train_data)*(1-val_split))
val_data: list = train_data[num_train_samples:]
print(len(val_data), " validation sentences")
train_data: list = train_data[:num_train_samples]
print(len(train_data), " training sentences")

Loading data from file data/train.tsv...
Loaded 395922 sentences
Splitting data...
39593  validation sentences
356329  training sentences


This is a utility function that will be used in a number of other functions. It takes a dictionary of
"counts" representing the counts of occurrences of different features and finds the probability of
each occurring in the specific context using the sum of all counts.

In [43]:
def compute_probabilities_from_counts(counts_dict: dict) -> dict:
    counts_sum: int = sum(counts_dict.values())
    probabilities_dict: dict = {}
    for count_id in counts_dict:
        count = counts_dict[count_id]
        probabilities_dict[count_id] = count / counts_sum
    assert round(sum(probabilities_dict.values()), 2) == 1.0, "All probabilities should sum to 1 but got {}".format(round(sum(probabilities_dict.values()), 2))
    return probabilities_dict

This is another utility function to be used later. It finds the maximum value in a dictionary and
returns its ID.

In [44]:
def get_max_float_id_from_dict(float_dict: dict) -> str:
    max_value: float = -0.1
    max_id: float = None
    for dict_id in float_dict:
        if float_dict[dict_id] > max_value:
            max_value = float_dict[dict_id]
            max_id = dict_id
    return max_id


This function determines the initial state probabilities of the provided data. It looks at only the
first word of each sentence and determines the probability of the first word in any sentence of
having each tag. For the training dataset the `N` state/tag was heavily favored, as expected.

These probabilities are later combined with the emission state probabilities when evaluating the
first word of a sentence, since the transition state cannot be evaluated with no previous state.

In [45]:
def generate_initial_state_probabilities(data: list) -> dict:
    initial_state_counts: dict = states.copy()
    for sentence in data:
        initial_state: str = sentence[0][1]
        if initial_state in initial_state_counts:
            initial_state_counts[initial_state] += 1
    initial_state_probabilities: dict = compute_probabilities_from_counts(initial_state_counts)
    return initial_state_probabilities

This function generates the probability of going from one state to another using the specified dataset. It does so with
a two-level dictionary. By iterating over each sentence but excluding the first (initial state) word,
the index provided by `enumerate` will be one behind each word's index in the sentence. Therefore this
becomes the `prev_idx` and can retrieve the previous state.

The first level of the dictionary is the preceding state and the second is the current state. We initially
count all occurrences, then use the above-defined `compute_probabilities_from_counts` function to turn these
into probabilities.

In [46]:
def generate_transition_state_probabilities(data: list) -> dict:
    # create a dictionary with two levels, the first being the previous state and the second being the current state
    transition_state_counts: dict = {state: states.copy() for state in states}
    for sentence in data:
        # since we enumerate over a list that excludes the first item, the enumeration index is one behind
        for prev_idx, word in enumerate(sentence[1:]):
            prev_state: str = sentence[prev_idx][1]
            current_state: str = word[1]
            if prev_state in transition_state_counts and current_state in transition_state_counts[prev_state]:
                transition_state_counts[prev_state][current_state] += 1
    transition_state_probabilities: dict = {state: {} for state in states}
    for prev_state in transition_state_counts:
        transition_state_probabilities[prev_state] = compute_probabilities_from_counts(transition_state_counts[prev_state])
    return transition_state_probabilities

This function generates the emission probability for each word given each state using another two-level dictionary.
It iterates over each word and counts the occurrence of word X in state Y's dictionary. It then
uses the `compute_probabilities_from_counts` function to get the **generative probabilities**. That is,
given a state/tag, how likely will that state/tag generate each word.

In [47]:
def generate_emission_probabilities(data: list) -> dict:
    emission_counts_by_state: dict = {state: {} for state in states}
    for sentence in data:
        for word_state_pair in sentence:
            word, state = word_state_pair
            if state in emission_counts_by_state:
                # initialize word in state dict if the first occurrence of word X in state Y
                if word not in emission_counts_by_state[state]:
                    emission_counts_by_state[state][word] = 0
                emission_counts_by_state[state][word] += 1
    emission_probabilities_by_state: dict = {state: {} for state in states}
    for state in emission_counts_by_state:
        emission_probabilities_by_state[state] = compute_probabilities_from_counts(emission_counts_by_state[state])
    return emission_probabilities_by_state

The HMM class has four methods:
- The `fit` method is run with `init` and simply runs all the probability-generating functions
defined above with the provided data.
- The `format_evaluation_data` method is used to convert the datasets from the format used
to fit a model into the format used by the `evaluate` method.
- The `predict` method is the most important part of the class. It takes a single sentence
as a string and uses the probability dictionaries to generate a predicted state sequence. It begins
by initializing `state_probabilities` as the `initial_state_probabilities`. This is because
there is no transition state probability with the first word. For all subsequent words `state_probabilities`
are initialized as the transition state probabilities based on the most likely previous state.
Each of the initial state / transition state probabilities are then multiplied by the emission probabilities.
*If a word hasn't appeared with any given state, its overall probability is set to 0. I know this isn't
smoothing and in an ideal situation / if I get time later it will be.* I tried a form of smoothing where it took
the minimum emission probability but this dramatically increased runtime.
- The `evaluate` method simply takes a list of sentences as strings and a list of lists of labels
as strings and runs predict on each sentence, comparing the outputs to the given labels and computing
accuracy as it goes.

In [48]:
class HMM:
    def __init__(self, data):
        self.initial_state_probabilities, self.transition_probabilities, self.emission_probabilities = self.fit(data)

    @staticmethod
    def fit(data: list) -> tuple:
        print("Fitting model to provided dataset...")
        initial_state_probabilities = generate_initial_state_probabilities(data)
        transition_probabilities = generate_transition_state_probabilities(data)
        emission_probabilities = generate_emission_probabilities(data)
        print("Model ready.")
        return initial_state_probabilities, transition_probabilities, emission_probabilities

    def format_data(self, evaluation_data: list) -> tuple:
        sentences: list = []
        labels: list = []
        for sentence in evaluation_data:
            sentences.append((" ".join(word_state_pair[0] for word_state_pair in sentence)))
            labels.append([word_state_pair[1] for word_state_pair in sentence])
        return sentences, labels

    def predict(self, sentence: str) -> dict:
        words: list = sentence.split(" ")
        state_sequence: list = []
        # begin each sentence using initial state probabilities, then switches to transition probabilities
        state_probabilities: dict = self.initial_state_probabilities.copy()
        for idx, word in enumerate(words):
            for state in state_probabilities:
                if word in self.emission_probabilities[state]:
                    state_probabilities[state] = state_probabilities[state]*self.emission_probabilities[state][word]
                else:
                    state_probabilities[state] = 0
            state_sequence.append(get_max_float_id_from_dict(state_probabilities))
            # for next word, initialize probabilities as transition probabilities from the previous state
            state_probabilities = self.transition_probabilities[state_sequence[idx]].copy()
        return state_sequence

    def evaluate(self, evaluation_data: list) -> float:
        sentences, labels = self.format_data(evaluation_data)
        total, correct = 0, 0
        print("Evaluating {} sentences...".format(len(sentences)))
        for sentence_idx, sentence in enumerate(sentences):
            predicted_sequence: list = self.predict(sentence)
            total += len(predicted_sequence)
            correct += sum([int(predicted == labels[sentence_idx][tag_idx]) for tag_idx, predicted in enumerate(predicted_sequence)])
        print("Done! {} correct tags out of {} words".format(correct, total))
        return correct / total

Create the model using the training data.

In [49]:
model = HMM(train_data)

Fitting model to provided dataset...
Model ready.


A couple use case sentences where the model picks up on the mutations.

In [50]:
val_sentence = " ".join([word[0] for word in val_data[100]])
print("Sentence:", val_sentence)
print("Predicted Sequence:", model.predict(val_sentence))
print("Ground Truth Sequence:", [word[1] for word in val_data[100]])

Sentence: d'fhan sé ina tost ar feadh scaithimh eile sular labhair sé .
Predicted Sequence: ['N', 'N', 'N', 'S', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
Ground Truth Sequence: ['N', 'N', 'N', 'S', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [51]:
val_sentence = " ".join([word[0] for word in val_data[1000]])
print("Sentence:", val_sentence)
print("Predicted Sequence:", model.predict(val_sentence))
print("Ground Truth Sequence:", [word[1] for word in val_data[1000]])

Sentence: mill an séasúr seo caite muid .
Predicted Sequence: ['S', 'N', 'N', 'N', 'N', 'N', 'N']
Ground Truth Sequence: ['S', 'N', 'N', 'N', 'N', 'N', 'N']


Evaluate validation set.

In [52]:
val_acc = model.evaluate(val_data)
print("Validation Accuracy: ", str(round(val_acc*100, 2)) + "%")

Evaluating 39593 sentences...
Done! 865363 correct tags out of 963502 words
Validation Accuracy:  89.81%


Evaluate testing set if specified.

In [55]:
if "test_data" in globals():
    test_acc = model.evaluate(test_data)
    print("Testing Accuracy: ", str(round(test_acc*100, 2)) + "%")
